<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Carlos Alberto Matias da Costa, 20013086' #@param {type:"string"}






In [5]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [101]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [106]:
host_counts = df['host_name'].value_counts()

top_3_hosts = host_counts.head(3)

top_3_hosts


host_name
Mark            7
Sonder (NYC)    7
Andrew          6
Name: count, dtype: int64

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [172]:
import pandas as pd

contagem_imoveis_por_host = df.groupby('host_id')['id'].count().reset_index()

contagem_imoveis_por_host.rename(columns={'id': 'numero_imoveis'}, inplace=True)

df = pd.merge(df, contagem_imoveis_por_host, on='host_id', how='left')

preco_mais_de_um_imovel = df[df['numero_imoveis'] > 1]['price'].mean()

preco_mais_um_imovel = df[df['numero_imoveis'] == 1]['price'].mean()

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

MergeError: Passing 'suffixes' which cause duplicate columns {'numero_imoveis_x'} is not allowed.

# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [111]:
import pandas as pd

Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
bound_baixa = Q1 - 1.5 * IQR
bound_alta = Q3 + 1.5 * IQR
manhattan_df = df[df['neighbourhood_group'] == 'Manhattan']
outliers_manhattan = manhattan_df[(manhattan_df['price'] < bound_baixa) | (manhattan_df['price'] > bound_alta)]
porcentagem_outliers = (len(outliers_manhattan) / len(manhattan_df)) * 100
print(f"Percentage of outliers in Manhattan: {porcentagem_outliers:.2f}%")

Percentage of outliers in Manhattan: 10.05%


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [110]:
import pandas as pd

def categorize_description(description):
    description = str(description).lower()
    if 'beautiful' in description:
        return 'beautiful'
    elif 'sunny' in description:
        return 'sunny'
    elif 'spacious' in description:
        return 'spacious'
    elif 'heart' in description:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_description)
word_stats = df.groupby('word').agg({'price': ['count', 'mean']})
word_stats


price            
          count        mean
word                       
beautiful    41  137.926829
heart        36  166.388889
others      624  157.886218
spacious     60  128.516667
sunny        43  137.674419

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [156]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes


id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [163]:
nova_tabela = pd.concat([df, df_taxes], axis=1)
nova_tabela
media_imposto = novo_dataframe.groupby(['neighbourhood_group','type'])['value'].mean()
media_imposto

neighbourhood_group  type 
Bronx                fines    1813.000000
                     taxes    1110.642857
Brooklyn             fines    1073.000000
                     taxes    1115.850394
Manhattan            fines     539.000000
                     taxes    1067.758993
Queens               fines     429.000000
                     taxes    1036.483333
Staten Island        fines            NaN
                     taxes    1196.200000
Name: value, dtype: float64

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [57]:

df_report = df[['neighbourhood_group', 'neighbourhood', 'price']].copy()

df_report = df_report.sort_values(by=['neighbourhood_group', 'neighbourhood'])

df_report['line_id'] = range(1, len(df_report) + 1)

page_id = 1
line_count = 0
current_neighbourhood_group = None
sum_price_pages = 0


page_ids = []
prices_list = []
neighbourhood_group_list = []
neighbourhood_list = []
line_ids = []

for index, row in df_report.iterrows():
    neighbourhood_group = row['neighbourhood_group']
    price = row['price']
    neighbourhood = row['neighbourhood']
    line_id = row['line_id']

    if neighbourhood_group != current_neighbourhood_group:
        if current_neighbourhood_group is not None:
            page_ids.append(page_id)
            prices_list.append(df_report[df_report['neighbourhood_group'] == current_neighbourhood_group]['price'].mean())
            neighbourhood_group_list.append(current_neighbourhood_group)
            neighbourhood_list.append('Group Mean')
            line_ids.append(line_id)

        page_id += 1
        line_count = 0
        current_neighbourhood_group = neighbourhood_group

    page_ids.append(page_id)
    prices_list.append(price)
    neighbourhood_group_list.append(neighbourhood_group)
    neighbourhood_list.append(neighbourhood)
    line_ids.append(line_id)

    line_count += 1

    if line_count == 30:
      page_id += 1
      line_count = 0

page_ids.append(page_id)
prices_list.append(df_report[df_report['neighbourhood_group'] == current_neighbourhood_group]['price'].mean())
neighbourhood_group_list.append(current_neighbourhood_group)
neighbourhood_list.append('Group Mean')
line_ids.append(line_id)


df_report = pd.DataFrame({
    'page_id': page_ids,
    'price': prices_list,
    'neighbourhood_group': neighbourhood_group_list,
    'neighbourhood': neighbourhood_list,
    'line_id': line_ids
})


In [82]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 11579


#### Aqui um *sample* do report.

In [83]:
df_report.iloc[-10::,:]

page_id      price neighbourhood_group  neighbourhood  line_id
6147      208  135.00000       Staten Island      Todt Hill     6144
6148      208  135.00000       Staten Island      Todt Hill     6145
6149      208  135.00000       Staten Island      Todt Hill     6146
6150      208  135.00000       Staten Island      Todt Hill     6147
6151      208  135.00000       Staten Island      Todt Hill     6148
6152      208  135.00000       Staten Island      Todt Hill     6149
6153      208  135.00000       Staten Island      Todt Hill     6150
6154      208  135.00000       Staten Island      Todt Hill     6151
6155      208   59.00000       Staten Island  Tompkinsville     6152
6156      208   93.77551       Staten Island     Group Mean     6152